In [1]:
# https://github.com/shoji9x9/CIFAR-10-By-small-ResNet/blob/master/ResNet-for-CIFAR-10-with-Keras.ipynb

In [2]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.constants import MODELS_FOLDER

In [3]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
import libs.model_archs
import libs.utils
from libs.seeds import load_model_seeds
model_seeds = load_model_seeds()

In [5]:
# define dataset and model architecture
dataset = "cifar10"
# Select ResNet Version
resnet_version = 1
model_arch = "resnetA" + str(resnet_version)

# set training hyperparameters
batch_size = 128
n_epochs = 200

In [6]:
# global seed
seed = model_seeds[0]
tf.random.set_seed(seed)
np.random.seed(seed)

# prepare data
dataset_loader = getattr(libs.utils, 'prepare_'+dataset)
to_categorical=False
(x_train, y_train), (x_test, y_test) = dataset_loader(to_categorical)

In [7]:
from tensorflow.keras.callbacks import Callback
class LearningController(Callback):
    def __init__(self, num_epoch=0, learn_minute=0):
        self.num_epoch = num_epoch
        self.learn_second = learn_minute * 60
        if self.learn_second > 0:
            print("Learning rate is controled by time.")
        elif self.num_epoch > 0:
            print("Learning rate is controled by epoch.")
        
    def on_train_begin(self, logs=None):
        if self.learn_second > 0:
            self.start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        if self.learn_second > 0:
            current_time = time.time()
            if current_time - self.start_time > self.learn_second:
                self.model.stop_training = True
                print("Time is up.")
                return

            if current_time - self.start_time > self.learn_second / 2:
                self.model.optimizer.lr = lr * 0.1            
            if current_time - self.start_time > self.learn_second * 3 / 4:
                self.model.optimizer.lr = lr * 0.01
                
        elif self.num_epoch > 0:
            if epoch > self.num_epoch / 2:
                self.model.optimizer.lr = lr * 0.1            
            if epoch > self.num_epoch * 3 / 4:
                self.model.optimizer.lr = lr * 0.01
                    
        print('\nlr:%.2e' % self.model.optimizer.lr.value())

In [8]:
# create model
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Activation, MaxPool2D, GlobalAveragePooling2D, Add, Input, Flatten
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2


input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
n_classes = y_train.shape[1]

# n_channels = x_train.shape[3] # no. of channels (?)
# # Computed depth of model
# if resnet_version == 1:
# 	depth = n_channels * 6 + 2
# elif resnet_version == 2:
# 	depth = n_channels * 9 + 2

# model_generator = getattr(libs.model_archs, model_arch)
# model = model_generator(input_shape, depth, n_classes)

# https://github.com/shoji9x9/CIFAR-10-By-small-ResNet/blob/master/ResNet-for-CIFAR-10-with-Keras.ipynb
n = 9 # 56 layers
channels = [16, 32, 64]

inputs = Input(shape=input_shape)
x = Conv2D(channels[0], 
           kernel_size=(3, 3), 
           padding="same", 
           kernel_initializer="he_normal", 
           kernel_regularizer=l2(1e-4))(inputs)
x = BatchNormalization()(x)
x = Activation(tf.nn.relu)(x)

for c in channels:
    for i in range(n):
        subsampling = i == 0 and c > 16
        strides = (2, 2) if subsampling else (1, 1)
        y = Conv2D(c, 
                   kernel_size=(3, 3), 
                   padding="same", 
                   strides=strides, 
                   kernel_initializer="he_normal", 
                   kernel_regularizer=l2(1e-4))(x)
        y = BatchNormalization()(y)
        y = Activation(tf.nn.relu)(y)
        y = Conv2D(c, 
                   kernel_size=(3, 3), 
                   padding="same", 
                   kernel_initializer="he_normal", 
                   kernel_regularizer=l2(1e-4))(y)
        y = BatchNormalization()(y)
        if subsampling:
            x = Conv2D(c, 
                       kernel_size=(1, 1), 
                       strides=(2, 2), 
                       padding="same", 
                       kernel_initializer="he_normal", 
                       kernel_regularizer=l2(1e-4))(x)
        x = Add()([x, y])
        x = Activation(tf.nn.relu)(x)

x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
outputs = Dense(10, 
                activation=tf.nn.softmax, 
                kernel_initializer="he_normal")(x)

model = Model(inputs=inputs, outputs=outputs)

model._name = "resnet_lrs_checkpnt" + str(6 * n + 2)

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = "ResNet-for-CIFAR-10-with-Keras.h5", 
                             monitor="val_loss", 
                             verbose=1, 
                             save_best_only=True)

In [10]:
# compile model
from tensorflow.keras.optimizers import SGD
# model.compile(loss ='categorical_crossentropy',
#               optimizer ='adam',
#               metrics =['accuracy'])
lr = 0.1
optimizer = SGD(learning_rate=lr, momentum=0.9)
model.compile(optimizer=optimizer, 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])

learning_controller = LearningController(n_epochs)
callbacks = [checkpoint, learning_controller]
# train model
model.fit(x_train, 
          y_train, 
          batch_size=batch_size, 
          epochs=n_epochs,
          validation_split=0.2,
          callbacks=callbacks)

Learning rate is controled by epoch.
Epoch 1/200
313/313 [==============================] - ETA: 0s - loss: 4.4672 - accuracy: 0.1679
Epoch 1: val_loss improved from inf to 4.01475, saving model to ResNet-for-CIFAR-10-with-Keras.h5

lr:1.00e-01
313/313 [==============================] - 72s 144ms/step - loss: 4.4672 - accuracy: 0.1679 - val_loss: 4.0148 - val_accuracy: 0.2044
Epoch 2/200
313/313 [==============================] - ETA: 0s - loss: 3.7334 - accuracy: 0.2652
Epoch 2: val_loss improved from 4.01475 to 3.91919, saving model to ResNet-for-CIFAR-10-with-Keras.h5

lr:1.00e-01
313/313 [==============================] - 41s 130ms/step - loss: 3.7334 - accuracy: 0.2652 - val_loss: 3.9192 - val_accuracy: 0.2430
Epoch 3/200
313/313 [==============================] - ETA: 0s - loss: 3.2723 - accuracy: 0.3722
Epoch 3: val_loss improved from 3.91919 to 3.46316, saving model to ResNet-for-CIFAR-10-with-Keras.h5

lr:1.00e-01
313/313 [==============================] - 42s 133ms/step - los

In [11]:
# evaluate model
score = model.evaluate(x_test, 
                       y_test, 
                       batch_size=batch_size)

print("Original Accuracy: ",score)

79/79 [==============================] - 3s 36ms/step - loss: 1.3214 - accuracy: 0.8043
Original Accuracy:  [1.321411371231079, 0.8043000102043152]


In [12]:
# save model
model_type = dataset + "--" + model_arch
model_instance = model_type + "-" + str(seed)
model_filename = model_instance + ".h5"
model_subdir = pathlib.Path(MODELS_FOLDER / model.name)
pathlib.Path(model_subdir).mkdir(parents=True, exist_ok=True)
model_file = str(pathlib.Path(model_subdir/ model_filename))
model.save(model_file)

In [13]:
model.summary()

Model: "resnet_lrs_checkpnt56"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_norma